In [26]:
import gdown
import zipfile
import os
import pandas as pd

Завантаження та розпакування файлу

In [27]:
url = "https://drive.google.com/uc?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO"
zip_path = "homework.zip"
data_path = "dataset/"

gdown.download(url, zip_path, quiet=False)
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(data_path)

data_folder = os.path.join(data_path, 'data')

activities = ['idle', 'running', 'stairs', 'walking']
file_list = [os.path.join(data_folder, activity, file) for activity in activities for file in os.listdir(os.path.join(data_folder, activity))]

data_frames = [pd.read_csv(file) for file in file_list]
data = pd.concat(data_frames, ignore_index=True)

print(data.head())
print(data.columns)

Downloading...
From: https://drive.google.com/uc?id=1nzrtQpfaHL0OgJ_eXzA7VuEj7XotrSWO
To: /content/homework.zip
100%|██████████| 4.21M/4.21M [00:00<00:00, 224MB/s]


   accelerometer_X  accelerometer_Y  accelerometer_Z
0         0.258574        -0.004788         9.749189
1         0.244209        -0.014365         9.773131
2         0.239420        -0.023942         9.787497
3         0.272939        -0.019154         9.782708
4         0.263362        -0.014365         9.787497
Index(['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z'], dtype='object')


In [13]:
!pip install tsfresh

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.1/169.1 kB 8.3 MB/s eta 0:00:00


In [29]:
from tsfresh.feature_extraction import feature_calculators

for column in data.columns:
    mean = feature_calculators.mean(data[column])
    std_dev = feature_calculators.standard_deviation(data[column])

    print(f"Time domain features for {column}:")
    print(f"Mean: {mean}")
    print(f"Standard Deviation: {std_dev}")
    print("\n")

Time domain features for accelerometer_X:
Mean: 1.9235497301660993
Standard Deviation: 8.40484551926619


Time domain features for accelerometer_Y:
Mean: 1.5983427839678117
Standard Deviation: 12.47400886847812


Time domain features for accelerometer_Z:
Mean: 1.8048960197152586
Standard Deviation: 7.19157161174301




In [31]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

data['labels'] = np.where(data['accelerometer_X'] > 0.5, 1, 0)

features = data[['accelerometer_X', 'accelerometer_Y', 'accelerometer_Z']]
labels = data['labels']


Нормалізація та розбиття на тренувальний і тестовий набори

In [32]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Створення моделей
svm_model = make_pipeline(StandardScaler(), SVC())
rf_model = RandomForestClassifier()

Навчання моделей

In [33]:
svm_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

RandomForestClassifier()

Оцінка результатів

In [34]:
# Для SVM
svm_predictions = svm_model.predict(X_test)
svm_accuracy = accuracy_score(y_test, svm_predictions)
svm_report = classification_report(y_test, svm_predictions)

# Для випадкового лісу
rf_predictions = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
rf_report = classification_report(y_test, rf_predictions)

print("SVM Accuracy:", svm_accuracy)
print("SVM Classification Report:\n", svm_report)

print("\nRandom Forest Accuracy:", rf_accuracy)
print("Random Forest Classification Report:\n", rf_report)

SVM Accuracy: 0.9972660682967089
SVM Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     22098
           1       1.00      0.99      1.00     16674

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772


Random Forest Accuracy: 1.0
Random Forest Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     22098
           1       1.00      1.00      1.00     16674

    accuracy                           1.00     38772
   macro avg       1.00      1.00      1.00     38772
weighted avg       1.00      1.00      1.00     38772

